## RL adapted to Mars Lander from Lunar Lander

#### From Mars Lander:

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive  # slider
from matplotlib import rcParams 
from numpy.linalg import norm
from numpy.random import randint
from ipywidgets import interactive 
from matplotlib import rcParams  

rcParams['figure.figsize'] = (10, 8)

In [3]:
def mars_surface():
    surfaceN = randint(5, 15)
    land = np.zeros((surfaceN, 2), dtype=int)
    
    # Ensure there's a flat landing site at least 1000m long
    landing_site = randint(1, surfaceN-1)
    land[landing_site, 0] = randint(2000, 5000)
    land[landing_site+1, 0] = min(land[landing_site, 0] + randint(1000, 2000), 6999)
    land[landing_site+1, 1] = land[landing_site, 1] = randint(1, 1500)
    
    # Fill in the rest of the terrain
    for i in range(landing_site):
        land[i, 0] = (land[landing_site, 0] / landing_site) * i
        land[i, 1] = randint(0, 1500)
    
    for i in range(landing_site + 2, surfaceN):
        land[i, 0] = (land[landing_site + 1, 0] + 
                      (7000 - land[landing_site + 1, 0]) / len(land[landing_site + 2:]) * 
                      (i - (landing_site + 1)))
        land[i, 1] = randint(0, 1500)
    
    # impose boundary conditions
    land[0, 0] = 0
    land[-1, 0] = 6999

    return land, landing_site

def plot_surface(land, landing_site):
    fig, ax = plt.subplots()
    ax.plot(land[:landing_site+1, 0], land[:landing_site+1, 1], 'k-')
    ax.plot(land[landing_site+1:, 0], land[landing_site+1:, 1], 'k-')
    ax.plot([land[landing_site, 0], land[landing_site+1, 0]], 
             [land[landing_site, 1], land[landing_site+1, 1]], 'k--')
    ax.set_xlim(0, 7000)
    ax.set_ylim(0, 16000) # HERE was 13000
    return ax

def plot_lander(land, landing_site, X, thrust=None, animate=False, step=10):
    if animate:
        def plot_frame(n=len(X)-1):
            ax = plot_surface(land, landing_site)
            ax.plot(X[:n, 0], X[:n, 1], 'b--')      # trajectory of lander
            ax.plot(X[n, 0], X[n, 1], 'k^', ms=20) # lander (color was b) , what is ^?
            if thrust is not None:
                ax.plot([X[n, 0], X[n, 0] - 100*thrust[n, 0]],
                        [X[n, 1] - 100., X[n, 1] - 100. - 100*thrust[n, 1]], 
                       'r-', lw=10)
        return interactive(plot_frame, n=(0, len(X), step)) #slider
    else:
        ax = plot_surface(land, landing_site) 
        ax.plot(X[:, 0], X[:, 1], 'b--')
        ax.plot(X[-1, 0], X[-1, 1], 'b^')
        return ax

def interpolate_surface(land, x):
    i,  = np.argwhere(land[:, 0] < x)[-1] # segment containing x is [i, i+1]
    m = (land[i+1, 1] - land[i, 1])/(land[i+1, 0] - land[i, 0]) # gradient
    x1, y1 = land[i, :] # point on line with eqn. y - y1 = m(x - x1) 
    return m*(x - x1) + y1

land, landing_site = mars_surface()

def height(land, X):
    return X[1] - interpolate_surface(land, X[0]) #1 in X[1] points to the vertical position y of the lander

assert abs(height(land, [1, land[0, 1]])) < 100.0 # height when on surface left edge should be close to zero
assert abs(height(land, [6999, land[-1, 1]])) < 100.0 # height when on surface at right edge should be close to zero

_land, _landing_site = mars_surface()

def _height(_land, X):
    return X[1] - interpolate_surface(_land, X[0])

points = np.zeros((10, 2))
points[:, 0] = randint(0, 7000, size=10)
points[:, 1] = randint(0, 16000, size=10)
for i in range(10):
    assert abs(height(_land, points[i, :]) - _height(_land, points[i, :])) < 1e-6

### Reinforcement Learning embedded with the simulation & autopilot functions

In [5]:
def rewardsystem(i,X,V,power,fuel,Xtarget,prev_shaping,total_reward,reward):
        """
    returns:
         s (list): The states
         r: the reward for that step
         done: True or False
    """
 
        # If lander moves away from landing target it loses reward back !         

        state = [
            X[0],              # s[0] is the horizontal coordinate
            X[1],              # s[1] is the vertical coordinate
            V[0],              # s[2] is the horizontal speed
            V[1],              # s[3] is the vertical speed
            #lander.angle,     # s[4] is the angle              # improvement - add angle (not the same as how much we want the lander to rotate) #angle = np.rad2deg(np.arctan(np.linalg.norm(V[:,0])/np.linalg.norm(V[:,1]))); # not sure at all
            #20.0 * lander.angularVelocity / FPS, # s[5] is the angular speed - improvement
            fuel
        ]
        assert len(state) == 5 # should be 7 with angle and angular speed

        shaping = (  # for rewards
            -10 * (state[0]-Xtarget) # negative reward decreases as the lander approches centre of the landing zone
            -10 * (state[1]-land[landing_site, 1]) # negative reward decreases as the lander approches the landing zone altitude
            - 10 * np.sqrt(state[2] * state[2] + state[3] * state[3])  # Magnitude of velocity vectors    # negative reward decreases as the velocity decreases  
            + 2 * fuel # reward decreases the decreasing fuel
        ) 
         
        if prev_shaping is not None:# from reset function, need it? put it in demo_heurisic?
            reward = shaping - prev_shaping         
        prev_shaping = shaping

        reward += (
            power * 0.0003   # might need to choose more appropriate reward numbers
        )  # less fuel spent is better

        
        return np.array(state, dtype=np.float32), reward, prev_shaping #, done, {}  

In [6]:
"""
        env (erased): 
        s (list): The state. Attributes:
                  s[0] is the horizontal coordinate
                  s[1] is the vertical coordinate
                  s[2] is the horizontal speed
                  s[3] is the vertical speed
                  s[4] is the angle
                  s[5] is the angular speed 
                  Could add extra states: fuel remaining

"""

g = 3.711 # m/s^2 , gravity on Mars
TSFC = 0.0003 # kg/(N*s)
Dc = 6.3525 # drag force as a function of velocity

def heuristic_simulation(X0, V0, land, landing_site, 
             fuel=400, dt=0.1, Nstep=1000, 
             autopilot=None, print_interval=100, parameters=None, parachute=None,prev_shaping = None,total_reward = 0,reward=0):
    
    n = len(X0)       # number of degrees of freedom (2 here)
    X = X0.copy()     # current position
    V = np.array(V0).astype(float)  #was X0.copy()     # current velocity
    Xs = np.zeros((Nstep, n)) # position history (trajectory) 
    Vs = np.zeros((Nstep, n)) # velocity history
    thrust = np.zeros((Nstep, n)) # thrust history
    drag = np.zeros((Nstep, n)) # drag history
    #angle = np.zeros((Nstep, n)) # should be 1 instead of n?
    success = False
    fuel_warning_printed = False
    rotate = randint(-90, 90)            # degrees, initial angle random (heading alignment phase)
    power = 0            # m/s^2, initial thrust power  
    e_prev = np.zeros(Nstep) # error history   

    for i in range(Nstep):
        Xs[i, :] = X     # Store positions
        Vs[i, :] = V     # Store velocities
        
        if autopilot is not None:
            
            # call to funtion 2
            
            rotate, power, parachute,total_reward,reward,prev_shaping = autopilot(i, X, V, fuel, rotate, power, parameters, parachute,dt,e_prev,Nstep,prev_shaping,total_reward,reward)
            assert abs(rotate) <= 90
            assert 0 <= power <= 12000
        
            rotate_rad = rotate * np.pi / 180.0 
            thrust[i, :] = power * np.array([np.sin(rotate_rad), 
                                             np.cos(rotate_rad)])
            if fuel <= 0: 
                if not fuel_warning_printed:
                    print("Fuel empty! Setting thrust to zero")
                    fuel_warning_printed = True
                thrust[i, :] = 0
            else:
                fuel -= TSFC * power * dt        
        m = 2600 + fuel  #kg , Mass of Lander + Rover + fuel  # fuel Mass loss
        
        if parachute == 0:                                               
            drag[i, :] = 0 # no Drag            
        else: # parachute == 1
            drag[i, :] = -Dc*np.linalg.norm(V)*V  # Drag - Parachute deployed                               
        A = np.array([0, -g]) + thrust[i, :]/m + drag[i, :]/m                         
        V += A * dt                          # update velocities
        X += V * dt                          # update positions
        
        
        #if i % print_interval == 0: 
            #print(f"i={i:03d} X=[{X[0]:8.3f} {X[1]:8.3f}] V=[{V[0]:8.3f} {V[1]:8.3f}]" # angle={angle:8.3f}
                  #f" thrust=[{thrust[i, 0]:8.3f} {thrust[i, 1]:8.3f}] fuel={fuel:8.3f} rotate={rotate:8.3f} parachute={parachute:8.3f}") 
       
        done = False 
        
        # check for safe or crash landing
        if X[1] < interpolate_surface(land, X[0]):
            if not (land[landing_site, 0] <= X[0] and X[0] <= land[landing_site + 1, 0]):
                print("crash! did not land on flat ground!")
                reward = -1000
                done = True
            elif rotate != 0:
                print("crash! did not land in a vertical position (tilt angle = 0 degrees)")
                reward = -1000
                done = True
            elif abs(V[1]) >= 27: #started as 40
                print("crash! vertical speed must be limited (<27m/s in absolute value), got ", abs(V[1]))
                reward -= -1000
                done = True
            elif abs(V[0]) >= 5: #started as 20
                print("crash! horizontal speed must be limited (<5m/s in absolute value), got ", abs(V[0]))
                reward -= -1000
                done = True
            else:
                print("safe landing - well done!")
                #success = True
                reward = +1000
                done = True
            Nstep = i
            total_reward += reward
            break
                 
    
    return done, total_reward

In [7]:
def proportional_autopilot(i, X, V, fuel, rotate, power,parameters, parachute, dt, e_prev, Nstep,prev_shaping,total_reward,reward):
    K_v,K_p,K_h,K_i,K_d = parameters
    
    c_v = 10.0 # target landing speed, m/s #c vertical
    c_h = 0 #c horizontal and vertical #trade-off for rotation to go back to 0
    
    # Height from landing platform 
    h = X[1]-land[landing_site, 1];
    # Horizontal displacement
    Xtarget = (land[landing_site+1, 0] + land[landing_site, 0]) // 2 
    dist = (Xtarget-X[0])  #X[i,0], pass the history of X (same for V)

    rotate = np.rad2deg(np.arctan2(dist,h-2000))   
    if h<2000:
        rotate = 0
      
    # Combine vertical & horizontal errors
    v_target_vert = -(c_v + K_v*(h-2000));
    v_target_horz = abs(c_h+K_h*dist)
    v_err_vert = abs(v_target_vert - V[1])
    v_err_horz = abs(v_target_horz - V[0])
    e =  v_err_vert + v_err_horz;
    e_d = 0
    if i>0:
        e_d = K_d*((e - e_prev[i-1])/dt)
    e_prev[i] = e     # Store error
    
    Pout = K_p*(e + e_d + K_i*(e_prev.sum()*dt)) 
    power = min(max(Pout, 0.0), 12000.0)   # max thrust
    
    if h > 10000:
        parachute = 0 
    else:
        parachute = 1 #open parachute
    
    
    # call funtion 3 for rewards
    
    s, r,prev_shaping = rewardsystem(i,X,V,power,fuel,Xtarget,prev_shaping,total_reward,reward)
    total_reward += r
    
    
    if i % 100 == 0:
        print(f'total_reward={total_reward:8.3f}','at step',i)  #e={e:8.3f} power={power:8.3f} K_p={K_p:8.3f} K_h={K_h:8.3f} K_v={K_v:8.3f} K_i={K_i:8.3f} K_d={K_d:8.3f}
        
    return (rotate, power, parachute,total_reward,reward,prev_shaping)

In [8]:
# ML demo_heuristic adapted:

def demo_heuristic_lander():
    
    total_reward = 0  #initialisation
    s = None # need to initialise states
    #prev_shaping = None
    
    while True:
    
        # call to funtion 1
        
        land, landing_site = mars_surface()
        K_p= 50.000 ; K_h= 0.005 ; K_v= 0.005 ; K_i= 0.005 ; K_d= 75.000
        iterations = 1
        for i in list(range(iterations)):
            X0 = [randint(2000, 5000), randint(15000, 16000)] 
            V0 = [randint(-50,50), randint(-500,-300)]
            try:
                done,total_reward = heuristic_simulation(X0, V0, land, landing_site, dt=0.1, Nstep=3500, 
                                                    autopilot=proportional_autopilot, fuel=400,parameters=[K_v,K_p,K_h,K_i,K_d],parachute=None,prev_shaping = None,total_reward = 0,reward = 0)
            except IndexError:
                print('Error: Out of bounds')
                continue
        
        #if steps % 20 == 0 or done:
            #print("observations:", " ".join([f"{x:+0.2f}" for x in s]))
            #print(f"step {steps} total_reward {total_reward:+0.2f}")
        #steps += 1
        
        if done:
            print('The total reward is:',total_reward)
            break
        
        plot_lander(land, landing_site, Xs, thrust, animate=True, step=10)

    return total_reward

In [ ]:
demo_heuristic_lander()